In [ ]:
!pip install uv
!uv pip install -q autogluon.timeseries --system
!uv pip uninstall -q torchaudio torchvision torchtext --system # fix incompatible package versions on Colab

In [ ]:
import pandas as pd
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [ ]:
data1=pd.read_csv('Bitcoin_pre_dataset_v4.csv',index_col=0)#Bitcoin_pre_dataset_v4_daily.csv time interval of one day

Basic：

In [ ]:
del data1['daily_avg_fees']
del data1['daily_avg_size']
del data1['bitcoin_hour_fees']#Not used

In [ ]:
data1.rename(columns={'Ave_tra_fee_rate': 'target'}, inplace=True)#Bitcoin price needs to be converted to 'Price'

In [ ]:
data1['item_id']='H1'

In [ ]:
data1new = data1.loc[:, ~data1.columns.isin(['Ord_total_fees', 'Total_Inscriptions', 'Ordinals_Inscription', 'hour_market_TotalV','hour_market_TotalT','cnt','Ord_Size_Usage','Ord_vSize_Usage','Ord_hour_fees'])]#Remove Ordinals related data

In [ ]:
data = TimeSeriesDataFrame.from_data_frame(
    data1,# or data1new
    id_column="item_id",
    timestamp_column="time"
)
data.head()

In [ ]:
prediction_length = 24#Set the prediction_length
#eval_metric=["MAE","RMSE","MAPE"]
train_data, test_data = data.train_test_split(prediction_length)

predictor = TimeSeriesPredictor(eval_metric="MAE",prediction_length=prediction_length).fit(
    train_data, presets="high_quality"
)

In [ ]:
predictor.leaderboard(test_data)

Graphics：

In [ ]:
modellist=['Chronos[base]', 'TemporalFusionTransformer']

In [ ]:
for mode in modellist:
  predictions = predictor.predict(train_data,model=mode)
  predictor.plot(
      data=data,
      predictions=predictions,
      max_num_item_ids=2,
      max_history_length=50,
  )

Fine tuning:

Refer to：https://github.com/amazon-science/chronos-forecasting/tree/main/scripts

In [ ]:
!pip install "chronos[training] @ git+https://github.com/amazon-science/chronos-forecasting.git"

Modify KERNEL_BANK in kernel-synth.py：

In [ ]:
KERNEL_BANK = [
    ExpSineSquared(periodicity=24 / LENGTH),  # H
    ExpSineSquared(periodicity=24 * 7 / LENGTH),  # H
    ExpSineSquared(periodicity=24 * 14 / LENGTH),  # H
    ExpSineSquared(periodicity=24 * 21 / LENGTH),  # H
    ExpSineSquared(periodicity=24 * 30 / LENGTH),  # H
    DotProduct(sigma_0=0.0),
    DotProduct(sigma_0=1.0),
    DotProduct(sigma_0=10.0),
    RBF(length_scale=0.1),
    RBF(length_scale=1.0),
    RBF(length_scale=10.0),
    RationalQuadratic(alpha=0.1),
    RationalQuadratic(alpha=1.0),
    RationalQuadratic(alpha=10.0),
    WhiteKernel(noise_level=0.1),
    WhiteKernel(noise_level=1.0),
    ConstantKernel(),
]

In [ ]:
!python kernel-synth.py \
    --num-series 30000 \ #The paper used 30,000

In [ ]:
CUDA_VISIBLE_DEVICES=0 python training/train.py --config /path/to/modified/config.yaml
# Fine-tuning after modifying the configuration
!python training/train.py --config /content/training/configs/config.yaml \
    --model-id amazon/chronos-t5-base \
    --no-random-init \
    --max-steps 1000 \
    --learning-rate 0.001

use !cp or huggingface for save

In [ ]:
from chronos import ChronosPipeline

pipeline = ChronosPipeline.from_pretrained("/content/output/run-0/checkpoint-final")
pipeline.model.model.push_to_hub("chronos-t5-base-fine-tuned")

results：

In [ ]:
import pandas as pd
from gluonts.dataset.common import ListDataset

static_features = data1.drop(columns=['time', 'target']).values

# start time
start = pd.Timestamp("2023-02-14 17:00:00")

# prediction_length
prediction_length = 24

feat_static_real = static_features

# Create training dataset
train_ds = ListDataset(
    [{"target": data2['target'].iloc[:-prediction_length].values, "start": start, "feat_dynamic_real": feat_static_real}],
    freq="1H" # Or the actual frequency of your data
)

# Create testing dataset
test_ds = ListDataset(
    [{"target": data2['target'].values, "start": start, "feat_dynamic_real": feat_static_real}],
    freq="1H" # Or the actual frequency of your data
)


In [ ]:
import numpy as np
import torch
from gluonts.dataset.split import split
from gluonts.ev.metrics import MAE,RMSE,MAPE
from gluonts.itertools import batcher
from gluonts.model.evaluation import evaluate_forecasts
from gluonts.model.forecast import SampleForecast
from tqdm.auto import tqdm

from chronos import ChronosPipeline

# Load dataset
batch_size = 32
num_samples = 20
prediction_length = 24

# Load Chronos
pipeline = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-small",# change fine-tuned model '/content/run-0/checkpoint-final'
    device_map="cuda:0",
    torch_dtype=torch.bfloat16,
)

# Split dataset for evaluation
_, test_template = split(test_ds, offset=-prediction_length)
test_data = test_template.generate_instances(prediction_length)

# Generate forecast samples
forecast_samples = []
for batch in tqdm(batcher(test_data.input, batch_size=32)):
    context = [torch.tensor(entry["target"]) for entry in batch]
    forecast_samples.append(
        pipeline.predict(
            context,
            prediction_length=prediction_length,
            num_samples=num_samples,
        ).numpy()
    )
forecast_samples = np.concatenate(forecast_samples)

# Convert forecast samples into gluonts SampleForecast objects
sample_forecasts = []
for item, ts in zip(forecast_samples, test_data.input):
    forecast_start_date = ts["start"] + len(ts["target"])
    sample_forecasts.append(
        SampleForecast(samples=item, start_date=forecast_start_date)
    )

# Evaluate
metrics_df = evaluate_forecasts(
    sample_forecasts,
    test_data=test_data,
    metrics=[
        MAE(),
        RMSE(),
        MAPE(),
    ],
)
metrics_df